In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# model
import lightgbm as lgb
# import xgboost as xg

# optz
from skopt.searchcv import BayesSearchCV
from skopt.space import Categorical, Real, Integer

# preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import KFold, train_test_split

# metrics 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
 

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# fillna 
train.MasVnrArea = train.MasVnrArea.fillna(0)
train.MasVnrType = train.MasVnrType.fillna('CBlock')
train.LotFrontage = train.LotFrontage.fillna(train.LotFrontage.median())
train.GarageYrBlt = train.GarageYrBlt.astype('string').fillna('NA').replace('.0', '')

In [4]:
cols = train.loc[:, train.isna().any()].columns.to_list()
train[cols] = train[cols].apply(lambda x: x.fillna('NA'))

In [5]:
y = train.SalePrice.astype(int)
X = train.drop(['SalePrice', 'Id'], axis=1)

In [6]:
X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,0,NA,NA,NA,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,MnPrv,NA,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,4,2010,WD,Normal


In [7]:
type_int = ['int16', 'int32', 'int64']
col_int = [i for i in X if X[i].dtypes in type_int]

type_flt = ['float16', 'float32', 'float64']
col_flt = [i for i in X if X[i].dtypes in type_flt]

col_cat = [i for i in X if X[i].dtypes not in type_flt and X[i].dtypes not in type_int]

In [8]:
train.shape

(1460, 81)

In [20]:
X.nunique() > 25

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea           True
Street           False
                 ...  
MiscVal          False
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
Length: 79, dtype: bool

In [29]:
X['LotFrontage'].dtype

dtype('O')

In [28]:
X['LotFrontage'] = X['LotFrontage'].astype(str).replace('.0', '')
X['LotArea'] = X['LotArea'].astype(str).replace('.0', '')

In [10]:
MSSubClass, 
MSZoning, 
Street, 
Alley, 
LotShape, 
LandContour,
Utilities, 
LotConfig, 
LandSlope, 
Neighborhood, 
Condition1, 
Condition2, 
BldgType, 
HouseStyle, 
OverallQual, 
OverallCond, 
RoofStyle,
RoofMatl, 
Exterior1st, 
Exterior2nd, 


NameError: name 'MSSubClass' is not defined